In [1]:

from alphafold3_pytorch.trainer import Trainer
from alphafold3_pytorch.alphafold3 import Alphafold3
from alphafold3_pytorch.inputs import PDBDataset
from alphafold3_pytorch.data.weighted_pdb_sampler import WeightedPDBSampler

2024-09-10 02:59:06.061 | INFO     | alphafold3_pytorch.tensor_typing:<module>:72 - Type checking is disabled.
2024-09-10 02:59:06.063 | INFO     | alphafold3_pytorch.tensor_typing:<module>:77 - Debugging is disabled.
[02:59:17] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[02:59:17] UFFTYPER: Unrecognized charge state for atom: 0
[02:59:17] UFFTYPER: Unrecognized atom type: Mn2+2 (0)
[02:59:17] UFFTYPER: Unrecognized atom type: Fe2+2 (0)
[02:59:17] UFFTYPER: Unrecognized atom type: Co3+3 (0)
[02:59:17] UFFTYPER: Unrecognized atom type: Ni3+2 (0)
[02:59:17] UFFTYPER: Unrecognized charge state for atom: 0
[02:59:17] UFFTYPER: Unrecognized atom type: Zn+2 (0)
[02:59:17] UFFTYPER: Unrecognized atom type: Ca+2 (0)


Loading CCD component SMILES strings from data/ccd_data/components_smiles.json.


/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/wangjun/conda_envs/af3/lib/python3.10/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/wangjun/conda_envs/af3/lib/python3.10/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/public/protein/workspace/wangjun/conda_envs/af3/lib/python3.10/site-packages/local_attention/rotary.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/cpfs01/projects-HDD/cfff-6f3a36a0cd1e_HDD/publ

In [9]:
alphafold3 = Alphafold3(
        dim_atom_inputs=10,
        dim_atompair_inputs=5,
        atoms_per_window=27,
        dim_template_feats=108,
        num_dist_bins=None,
        confidence_head_kwargs=dict(pairformer_depth=1),
        template_embedder_kwargs=dict(pairformer_stack_depth=1),
        msa_module_kwargs=dict(depth=1),
        pairformer_stack=dict(depth=2),
        diffusion_module_kwargs=dict(
            atom_encoder_depth=1,
            token_transformer_depth=1,
            atom_decoder_depth=1,
        ),
        
        # jwang's debug parameters
        # dim_token=128,
        
    )

In [10]:

num_parameters = sum(p.numel() for p in alphafold3.parameters())
print(num_parameters)

31183688


In [13]:
from omegaconf import OmegaConf
conf = OmegaConf.load('tests/configs/alphafold3.yaml')
print(conf)

alphafold3 = Alphafold3(
    **conf
)
num_parameters = sum(p.numel() for p in alphafold3.parameters())
print(num_parameters)

{'dim_atom_inputs': 10, 'dim_template_feats': 108, 'dim_template_model': 64, 'atoms_per_window': 27, 'dim_atom': 128, 'dim_atompair_inputs': 5, 'dim_atompair': 16, 'dim_input_embedder_token': 384, 'dim_single': 384, 'dim_pairwise': 128, 'dim_token': 768, 'ignore_index': -1, 'num_dist_bins': None, 'num_plddt_bins': 50, 'num_pde_bins': 64, 'num_pae_bins': 64, 'sigma_data': 16, 'diffusion_num_augmentations': 4, 'loss_confidence_weight': 0.0001, 'loss_distogram_weight': 0.01, 'loss_diffusion_weight': 4.0}
487451062


487451062


In [14]:
!nvidia-smi

Tue Sep 10 03:08:25 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:19:00.0 Off |                    0 |
| N/A   34C    P0    73W / 400W |    555MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:48:00.0 Off |                    0 |
| N/A   

In [15]:
alphafold3 = alphafold3.to('cuda')
!nvidia-smi

Tue Sep 10 03:08:30 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 12.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:19:00.0 Off |                    0 |
| N/A   34C    P0    73W / 400W |   2379MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:48:00.0 Off |                    0 |
| N/A   